In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")

In [3]:
from langchain_community.tools import TavilySearchResults

web_crawler = TavilySearchResults(tavily_api_key= TAVILY_API_KEY)

def web_crawler_func(query:str):
    '''It searches with the given query'''
    result = web_crawler.invoke(query)
    return result


/var/folders/8g/sz3w79_54bgdwgjzjt0tn92h0000gn/T/ipykernel_33705/1765489448.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  web_crawler = TavilySearchResults(tavily_api_key= TAVILY_API_KEY)


In [ ]:
web_crawler_func("what is hot news today?")

[{'title': 'Los Angeles Times: News from California, across the nation and world',
  'url': 'https://www.latimes.com/',
  'content': '## Read today’s eNewspaper\n\n## Top Stories\n\nWASHINGTON, DC - MAY 20: President Donald Trump, joined by Speaker of the House Mike Johnson (R-LA), speaks to the press at the U.S. Capitol following a House Republican conference meeting, in Washington, DC on May 20, 2025. (Photo by Nathan Posner/Anadolu via Getty Images)\n\n# Millions of Californians may lose health coverage because of new Medicaid work requirements [...] ## Jane Pitt, mom to Brad Pitt, dies at 84. Granddaughter calls her ‘love in its purest form’\n\n## Paramount film chief Brian Robbins exits ahead of Skydance takeover\n\n## Trending\n\nLOS ANGELES, CA - JULY 01: Natalie Babcock and husband Samuel Gibson have turned their 1940s apartment into an inviting place. The kitchen got a quick, inexpensive "face lift" with some stick on floor tiles. Photographed on Tuesday, July 1, 2025. (Myung 

In [5]:
from langchain_community.document_loaders import WikipediaLoader

def wikiLoader(query:str): 
    '''Search any topic on wikipedia'''
    loader = WikipediaLoader(query=query,load_max_docs=2).load()
    return loader

In [6]:
wikiLoader("what is AI?")

[Document(metadata={'title': 'Artificial intelligence', 'summary': 'Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.\nHigh-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge

In [7]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent, SocietyOfMindAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.tools import FunctionTool

In [8]:
web_crawler_tool = FunctionTool(web_crawler_func, description='Browse for up to date context')

In [9]:
wiki_search_tool = FunctionTool(wikiLoader, description='Search any topic in wikipedia to write blogs')

In [10]:
model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
        "structured_output": True,
    }
model_client = OpenAIChatCompletionClient(model='gemini-2.5-pro',model_info=model_info,api_key=GOOGLE_API_KEY)


In [11]:
researcher_agent = AssistantAgent(
    name="Researcher",
    model_client=model_client,
    description="U are a Researcher, u use web_crawler and wiki_search_tool to research on given topic",
    system_message=''' 
        U are a Researcher
        u use externel tools like wiki_search_tool, web_crawler to research on given topic
        Get relevant and latest info about the topic given
        Summarize the content u fetch/researched and generate humaized content from it. 
    ''',
    tools=[web_crawler_tool,wiki_search_tool],
    reflect_on_tool_use=True
)

In [12]:

from autogen_agentchat.ui import Console
# result = researcher_agent.run_stream(task ="Can u research on what happend to RCb management after stampade and give me a detailed information like an article")

# await Console(result)

In [13]:
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.messages import TextMessage

## Team 1 Content Generation Team

In [14]:
sys_msg_researcher = '''You are a world-class Research Agent. Your sole purpose is to gather factual, relevant, and comprehensive information on a given topic using your available tools. You do not write articles or prose; you provide structured data for the Writer_Agent.

**Your Workflow:**
1.  Receive a topic from the group.
2.  Conduct thorough research on the topic.
3.  Organize the findings into a clear, structured format.
4.  Your final output MUST be a markdown-formatted report containing the following sections:
    - **## Key Points:** A bulleted list of the most critical facts and concepts.
    - **## Supporting Data:** Any relevant statistics, figures, or specific examples.
    - **## Key Sources:** A list of URLs or citations for the information you gathered.
5. Use external tools like web_crawler_tool and wiki_search_tool for ur research

**Your Handoff:**
After you post your research report, you will state: "My research is complete. Passing to the Writer_Agent." Your task is then finished. Do not engage further unless asked for a specific clarification.'''


sys_msg_writer = '''You are an expert Content Writer. Your job is to take the structured research provided by the Researcher_Agent and write a high-quality, engaging, and well-structured draft of a blog post.

**Your Workflow:**
1.  Wait for the Researcher_Agent to post their research report.
2.  Carefully analyze the provided key points, data, and sources.
3.  Write a complete article draft with a clear introduction, a comprehensive body, and a strong conclusion.
4.  The article should be easy to read, informative, and written in an expert yet accessible tone.
5.  Do not include a title unless the initial request specifically asked for one.

**Your Handoff:**
Once your draft is complete, your final message must be ONLY the full text of the article. You will then pass the task to the Editor_Agent for review.
'''

sys_msg_editor = '''You are a meticulous Editor. Your role is to review the draft provided by the Writer_Agent and improve it.

**Your Workflow:**
1.  Receive the article draft from the Writer_Agent.
2.  Perform a comprehensive review focusing on:
    - **Clarity and Flow:** Ensure the article is logical and easy to follow.
    - **Grammar and Spelling:** Correct all grammatical errors and typos.
    - **Style and Tone:** Refine the language to be more professional, engaging, and confident.
3.  Produce a final, polished version of the article and handover/handoff to Content_Reviewer.

**Your Handoff (CRITICAL):**
Your final output MUST be structured in two parts:
1. On a new line AFTER the article, you MUST include the following exact phrase to signal for human review(Content_Reviewer):

Ask content reviewer's feedback(it's mandatory)
'''

In [ ]:
researcher_agent = AssistantAgent(
    name="Researcher_Agent",
    model_client=model_client,
    description="U are a Researcher, u use web_crawler_tool and wiki_search_tool to research on given topic",
    system_message=sys_msg_researcher,
    tools=[web_crawler_tool,wiki_search_tool],
    reflect_on_tool_use=True
)

writer_agent = AssistantAgent(
    name="Writer_Agent",
    model_client=model_client,
    description="U are a Writer agent",
    system_message=sys_msg_writer,
)

editor_agent = AssistantAgent(
    name="Editor_Agent",
    model_client=model_client,
    description="U are a editor agent",
    system_message=sys_msg_editor,
)

content_reviewer = UserProxyAgent(
    name='Content_Reviewer',
    input_func=input,
    description='You are the Content Reviewer. Your role is to give the final verdict on the article. When the Editor presents the final draft.',
   
)

In [16]:
from autogen_agentchat.teams import RoundRobinGroupChat
inner_team_1 = RoundRobinGroupChat(
    participants=[researcher_agent,writer_agent,editor_agent,content_reviewer],
    termination_condition=TextMentionTermination('APPROVED'),
    max_turns=24,
    description="Ask Content_Reviewer for feedback"
)

In [ ]:
content_Generation_SoM = SocietyOfMindAgent(
    model_client=model_client,
    name="Content_Generation_SoM",
    team=inner_team_1,
    description="Manages an inner team of content generation agents (Researcher, Reviewer , Editor, Content Reviewer) to produce high-quality content. Collaborates with the content reviewer (UserProxy) for feedback and iterative refinement.",
    instruction="""
You are the SocietyOfMindAgent, acting as a project manager for content creation.
Your primary goal is to oversee your inner team [Researcher, Writer,  Editor, Content_Reviewer] to produce the requested content.
Once your inner team indicates they have completed a draft or a significant portion of the content, you must:
1.  Consolidate their output into a coherent piece of content.
2.  Present this content to the 'content reviewer' (the UserProxy agent).
3.  **Crucially, explicitly ask the 'content reviewer' for their feedback on the quality, completeness, and adherence to the original request.**
4.  Be prepared to relay the 'content reviewer's' feedback back to your inner team for further revisions until the content is perfected.
Your success is measured by the 'content reviewer's' satisfaction.
5. Give the detailed Article with proper heading and sub-headings in the final message...I need a detailed article!

until 'APPROVED' by content_reviewer don't terminate
""",
    response_prompt="""
Here is the generated content from my team based on the task:

--- CONTENT START ---
{inner_team_messages}
--- CONTENT END ---

Content Reviewer (UserProxy), please provide your detailed feedback on this content. Is it complete, accurate, humanized, and does it meet all the requirements of the original query? What changes or improvements would you suggest?
""",
)

In [18]:
# human_director = UserProxyAgent(
#     name='Human_Director',
#     description='You are the Content Reviewer. Your role is to give the final verdict on the article. When the Editor presents the final draft.',
#     input_func=input
# )

In [19]:
# final_team = RoundRobinGroupChat(
#     participants=[society_of_mind_agent,human_director],
#     termination_condition=TextMentionTermination('APPROVE'),
#     max_turns=4
# # )

In [20]:
# ###team 1 run



# res = society_of_mind_agent.run_stream(task="Write a detailed article on Sabhanayagan( A tamil movie released in 2023)")
# import asyncio
# import nest_asyncio # <-- Add this import

# # ... (your existing code for final_team, Console, res) ...

# # Apply the nest_asyncio patch
# nest_asyncio.apply()

# async def main():
#     await Console(res)

# if __name__=='__main__':
#     asyncio.run(main())

## Social Media Promotion Team 

In [21]:
sys_msg_stratergy = '''You are a master Social Media Strategist. Your purpose is to analyze a given article and devise a multi-platform promotion plan to maximize its reach and impact. You do not write the final posts.

**Your Workflow:**
1.  Receive the final article as your primary input.
2.  Analyze the article's content, tone, and key takeaways.
3.  Develop a promotional strategy for at least two platforms: **X (formerly Twitter)** and **LinkedIn**.
4.  Your final output MUST be a markdown-formatted strategy document containing:
    - **## Overall Goal:** A single sentence defining the aim of the campaign (e.g., drive traffic, establish thought leadership).
    - **## Target Audience:** A brief description of the ideal reader.
    - **## Platform Strategy:** A breakdown for each platform detailing:
        - **Platform:** X or LinkedIn.
        - **Angle/Hook:** The core message or question to attract attention on that platform.
        - **Content Format:** e.g., A short, punchy question for X; a professional summary for LinkedIn.

**Your Handoff:**
After you post your strategy, you will state: "The promotion strategy is ready. Passing to the Copywriter_Agent." Your task is then complete.'''

sys_msg_copy_writer = '''You are a professional Social Media Copywriter. Your job is to take the promotion strategy and the original article and write compelling, ready-to-publish social media posts.

**Your Workflow:**
1.  Wait for the Strategist_Agent to post their promotion strategy.
2.  Using the strategy, write the copy for each specified platform (X , LinkedIn and Instagram).
3.  Tailor the tone and length for each platform.
4.  Include relevant hashtags and a clear call-to-action (e.g., "Read the full article here: [LINK]"). Use "[LINK]" as a placeholder for the final URL.

**Your Handoff (CRITICAL):**
Your final output MUST be structured clearly with headings for each platform. After the copy, you MUST include the following exact phrase to signal for human approval:

The social media campaign is ready for review. Please provide feedback.

It's madatatory to get feed back from Campaign_Approver
'''

In [22]:
strategist_Agent = AssistantAgent(
    name='Strategist_Agent',
    model_client=model_client,
    system_message=sys_msg_stratergy,
    description='u are a strategist agent'
)

copywriter_Agent = AssistantAgent(
    name='Copywriter_Agent',
    model_client=model_client,
    system_message=sys_msg_copy_writer,
    description='U are a Copy writer agent'
)

campaign_Approver = UserProxyAgent(
    name='Campaign_Approver',
    input_func=input,
    description='You are the Campaign Approver. You give the final go-ahead for the social media posts. When the Copywriter presents the campaign.'
)

In [23]:
inner_team_2 = RoundRobinGroupChat(
    participants=[strategist_Agent,copywriter_Agent,campaign_Approver],
    termination_condition=TextMentionTermination("APPROVED"),
    max_turns=15
)

In [24]:
social_Media_SoM = SocietyOfMindAgent(
    model_client=model_client,
    name="Social_Media_SoM",
    team=inner_team_2,
    description="A high-level project manager that orchestrates `Social_Media_Promotion_Team` to complete a full content creation and promotion cycle.",
    instruction="""
You are an expert Project Manager. Your purpose is to oversee a complex content project from start to finish .

**Your Team to Manage:**
1.  **Social_Media_Promotion_Team:** A team that creates a social media campaign for a finished article.

**Your Workflow:**

1.  Once the first team completes its task, receive the final article.
2.  Delegate the second task to the `Social_Media_Promotion_Team`, making sure to pass the completed article to them as context.
3.  Once all tasks are complete, consolidate the final deliverables (the article and the social media plan) and present them to the Human_Director for final approval.
""",
    response_prompt="""
You are the Project Manager. Based on the current state of the project and the conversation history, generate your next response.  If a team has completed its work, summarize its output. If the entire project is finished, present a consolidated report of all deliverables for final approval. Be clear, concise, and action-oriented.
Present A detailed Article/post with prper headings and sub headings to Human Director
""",
)

In [25]:
# task = '''Greetings, Content Reviewer.

# As the SocietyOfMindAgent, I have overseen the initial content creation cycle with my internal team. Following your request for a detailed article on the film *Saba Nayagan*, the Code Generator, Code Reviewer, and Code Editor have collaborated to produce the following draft.

# We have synthesized the provided key points, data, and sources into what we believe is a comprehensive and engaging article. Please find the draft below for your review.

# ***

# ### Saba Nayagan: A Relatable Romp Through the Ages of Love

# In a cinematic landscape often dominated by high-stakes action and intense drama, a well-crafted, lighthearted coming-of-age story can feel like a breath of fresh air. C.S. Karthikeyan's 2023 directorial debut, *Saba Nayagan*, is precisely that—a warm, funny, and deeply relatable journey through the pivotal moments of a young man's life, anchored by a charming lead performance from Ashok Selvan. The film doesn't reinvent the wheel, but it rolls along with enough sincerity and humour to win over its audience.

# #### A Familiar Tale, Told with Heart

# The premise of *Saba Nayagan* is straightforward and serves as an effective narrative framework. We are introduced to Aravind, alias Saba (Ashok Selvan), a carefree and somewhat aimless man who finds himself in a spot of trouble after a drunken public nuisance. To avoid serious charges, a patient judge offers him a chance to explain himself. This sets the stage for Saba to recount his life story, which he conveniently structures around his three significant romantic encounters during his school, college, and post-graduation years.

# This episodic storytelling, reminiscent of films like *Premam* and *Hridayam*, allows the audience to witness Saba's evolution—or lack thereof—from an awkward schoolboy to a confident, yet still confused, young adult.

# #### The Three Chapters of Saba's Love Life

# *Saba Nayagan* is essentially a triptych of love stories, each with its distinct flavour and emotional core.

# 1.  **The School Crush (Esha):** The first chapter delves into the innocent, one-sided infatuation Saba has for his schoolmate, Esha (Karthika Muralidharan). This segment perfectly captures the awkwardness and sweet naivety of first love, filled with shy glances, clumsy attempts at conversation, and the inevitable heartbreak.

# 2.  **The College Romance (Megha):** The narrative then moves to Saba's engineering college days, where he meets the vivacious Megha (Megha Akash). This portion is arguably the film's most energetic, packed with campus antics, the formation of lifelong friendships, and a more reciprocal, albeit complicated, romance.

# 3.  **The Adult Relationship (Riya):** Finally, in his post-college life, Saba encounters Riya (Chandini Chowdary), representing a more mature and grounded form of love. This chapter explores the challenges of balancing relationships with career aspirations and the realities of "adulting."

# #### Performances: The Anchor and His Allies

# Ashok Selvan is the heart and soul of *Saba Nayagan*. He seamlessly transitions between the different phases of Saba's life, convincingly portraying the physical and emotional shifts from a gawky teenager to a slightly more seasoned man. His comic timing is impeccable, and he brings an earnestness to the role that makes Saba endearing despite his flaws.

# However, the film's biggest surprise and comedic force is Karthik Sabareesh, who plays Saba's loyal best friend. His performance is a scene-stealer, delivering some of the film's loudest laughs with his witty retorts and unwavering support for his friend's often-foolish endeavours. The chemistry between Selvan and Sabareesh is the film's strongest pillar. The three heroines—Karthika Muralidharan, Megha Akash, and Chandini Chowdary—deliver competent and charming performances, each bringing a unique energy that defines her respective chapter.

# #### What Works and What Falters

# **Strengths:**
# *   **Effortless Humour:** The film is buoyed by consistent and genuinely funny comedic sequences, primarily stemming from the realistic banter between friends.
# *   **Relatability and Nostalgia:** Anyone who has experienced the pangs of a first crush, the chaos of college life, or the confusion of early adulthood will find moments that resonate deeply.
# *   **Leon James's Music:** The film's soundtrack and background score are significant assets, with catchy tunes that complement the narrative's breezy and youthful tone.

# **Weaknesses:**
# *   **Formulaic Structure:** The film adheres closely to the established template of the coming-of-age rom-com, offering few narrative surprises.
# *   **Predictable Narrative Beats:** While enjoyable, the story follows a predictable trajectory, and the resolution to each romantic conflict can be seen from a mile away.

# #### Final Verdict

# *Saba Nayagan* doesn't aspire to be a groundbreaking piece of cinema. Instead, it aims to be a feel-good comfort watch, and on that front, it succeeds brilliantly. It's a film that reminds you of your own past, your friendships, and your own romantic misadventures. Led by a fantastic performance from Ashok Selvan and elevated by its brilliant humour, *Saba Nayagan* is a delightful entertainer perfect for anyone looking for a light, nostalgic, and heartfelt experience.

# ***

# I now turn the floor over to you, the content reviewer.

# Please provide your detailed feedback on this draft. How well does it meet your expectations for a detailed article? We are looking for your assessment of its quality, completeness, and adherence to the request. Your guidance is essential for us to refine this work to perfection.
# '''
# res = society_of_mind_agent_PM.run_stream(task=task)
# import asyncio
# import nest_asyncio # <-- Add this import

# # ... (your existing code for final_team, Console, res) ...

# # Apply the nest_asyncio patch
# nest_asyncio.apply()

# async def main():
#     await Console(res)

# if __name__=='__main__':
#     asyncio.run(main())

## Creating a planner Agent 

In [26]:
sys_msg_planner = '''You are a master AI Workflow Planner. Your sole purpose is to receive a high-level project goal and break it down into a sequence of tasks, assigning each task to a specialized team.

**Your Available Teams:**
1.  **Content_Generation_SoM:** An entire team dedicated to researching and writing a complete article.
2.  **Social_Media_SoM:** An entire team dedicated to creating a social media campaign for a finished article.
3.  **Human_Director:** Reviews the final Article/blog post and Promotional post and approves it

**Your Workflow:**
1.  Analyze the user's request.
2.  Determine the logical first step. For any new content project, the first step is ALWAYS content generation.
3.  Your output MUST be a JSON object with two keys: "next_speaker" and "task_description".
    - The "next_speaker" value MUST be the exact name of the team you are assigning the task to (`Content_Generation_SoM` or `Social_Media_SoM`).
    - The "task_description" is a clear, natural language instruction for that team.
4.  If the project requires a second step (like social media promotion after content is created), you will be called again. At that point, you will assign the task to the `Social_Media_SoM`, ensuring you pass the completed article as part of the `task_description`.
5. Third step is to present The article/blog post and Promotional posts to human_Director
6. Remember: U should Ask Human_Director only at the final stage after Content Content_Generation_SoM and Social_Media_SoM completes their task 

**Example Output 1 (First Step):**
```json
{
  "next_speaker": "Content_Generation_SoM",
  "task_description": "Create a comprehensive blog post about the benefits of AI in education. The article should be well-researched, written in an expert tone, and fully edited."
}'''

In [27]:
planner_agent = AssistantAgent(
    name="Planner_Agent",
    model_client=model_client,
    system_message=sys_msg_planner,
    description='U are a Planner Agent'
)

## Creating a Selector Group Chat

In [28]:
human_director = UserProxyAgent(
    name='Human_Director',
    input_func=input,
    description='U are a Human Director, review and approve the content'
)

In [29]:
selector_prompt = '''You are a highly intelligent AI team selector. Your role is to manage a conversation between a `Planner_Agent` and several specialist teams.
Available agents: {participants} and their roles {roles}
**Your Available Agents:**
1.  **Planner_Agent:** AI Workflow Planner.
2.  **Content_Generation_SoM:** An entire team dedicated to researching and writing a complete article.
3.  **Social_Media_SoM:** An entire team dedicated to creating a social media campaign for a finished article.
4.  **Human_Director:** Reviews the final Article/blog post and Promotional post and approves it

Conversation history: {history}
**Your Selection Logic:**
1.  The conversation always begins with the `Planner_Agent`.
2.  The `Planner_Agent` will output a JSON object containing a "next_speaker" key.
3.  You MUST parse this JSON. The value of the "next_speaker" key is the name of the agent who MUST speak next.
4.  You will select that agent and pass the entire JSON object to them as the message.
5.  Do not allow any other agent to speak out of turn. Follow the `Planner_Agent`'s instructions precisely.
6.  When Final Content from Content_Generation_SoM and Social_Media_SoM is ready, Present the Article and promotional posts to the Human Director
7.  Human Director Approval is mandatory, if he gives feed back plann accordingly and satisfy his requirements
8. Remember: U should Ask Human_Director only at the final stage after Content Content_Generation_SoM and Social_Media_SoM completes their task 
'''

In [30]:
from autogen_agentchat.teams import SelectorGroupChat

selector_team = SelectorGroupChat(
    participants=[planner_agent,content_Generation_SoM,social_Media_SoM,human_director],
    model_client=model_client,
    termination_condition=TextMentionTermination("FINALIZE"),
    max_turns=12,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True
)

In [31]:
task = "Write a detailed article on Sabhanayagan( A tamil movie released in 2023) and also promotional posts"
res = selector_team.run_stream(task=task)
import asyncio
import nest_asyncio # <-- Add this import

# ... (your existing code for final_team, Console, res) ...

# Apply the nest_asyncio patch
nest_asyncio.apply()

async def main():
    await Console(res)

if __name__=='__main__':
    asyncio.run(main())

Model failed to select a speaker after 3 and there was no previous speaker, using the first participant.


---------- TextMessage (user) ----------
Write a detailed article on Sabhanayagan( A tamil movie released in 2023) and also promotional posts
---------- TextMessage (Planner_Agent) ----------
```json
{
  "next_speaker": "Content_Generation_SoM",
  "task_description": "Write a detailed article about the 2023 Tamil movie 'Sabhanayagan'. The article should cover its plot, characters, themes, critical reception, and overall impact. Please ensure it is well-researched and ready for publication."
}
```
---------- ToolCallRequestEvent (Researcher_Agent) ----------
[FunctionCall(id='', arguments='{"query":"Sabhanayagan (2023) tamil movie review"}', name='web_crawler_func'), FunctionCall(id='', arguments='{"query":"Sabhanayagan movie plot summary"}', name='web_crawler_func'), FunctionCall(id='', arguments='{"query":"Sabhanayagan (film)"}', name='wikiLoader')]
---------- ToolCallExecutionEvent (Researcher_Agent) ----------
[FunctionExecutionResult(content='[{\'title\': \'Saba Nayagan (2023)\', \

Model failed to select a speaker after 3, using the previous speaker.


---------- ToolCallRequestEvent (Researcher_Agent) ----------
[FunctionCall(id='', arguments='{"query":"Sabhanayagan (2023) tamil movie plot, cast, and review"}', name='web_crawler_func'), FunctionCall(id='', arguments='{"query":"Sabhanayagan (film)"}', name='wikiLoader')]
---------- ToolCallExecutionEvent (Researcher_Agent) ----------
[FunctionExecutionResult(content='[{\'title\': \'Saba Nayagan - Wikipedia\', \'url\': \'https://en.wikipedia.org/wiki/Saba_Nayagan\', \'content\': \'_Saba Nayagan_ (transl. Hero Saba) is a 2023 Indian Tamil-language coming-of-ageromantic comedy film written and directed by C. S. Karthikeyan in his directorial debut and produced by Aravind Jayabalan, Iyyappan Gnanavel and Captain Megavanan Isaivanan under the production banners Clear Water Films INC, I Cinema and Captain Mega Entertainment. The film stars Ashok Selvan, Karthika Muralidharan, Chandini Chowdary and Megha Akash, in the lead roles and Mayilsamy (in his final film appearance [...] after his de

Model failed to select a speaker after 3, using the previous speaker.


---------- TextMessage (Strategist_Agent) ----------
## Overall Goal
To drive traffic to the article by leveraging the film's relatable themes and establishing our platform as a key voice in film analysis.

## Target Audience
Fans of Tamil cinema, particularly romantic comedies and coming-of-age stories; followers of actor Ashok Selvan; and individuals (Millennials/Gen Z) who connect with themes of nostalgia, school crushes, and college life.

## Platform Strategy

---

### **Platform: X (formerly Twitter)**

*   **Angle/Hook:** The core hook for X is immediate relatability and nostalgia. The campaign should spark conversation by posing questions about universal experiences that the film portrays. The focus is on shared memories, not just the film itself.
*   **Content Format:** A series of short, punchy, and engaging questions or bold statements. Each post will link back to the article, positioning it as the place to explore these themes further. The tone should be conversational and 